<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/4_AgrupamentoClassificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import ClusteringEvaluator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
pagamentos_pedido_preprocessados = spark.read.option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido_preprocessados_parquet')
joins_itens_pedido = spark.read.option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/joins_itens_pedido_parquet')
produtos_preprocessados = spark.read.option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos_preprocessados_parquet')
avaliacoes_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pedidos.csv', header=True, inferSchema=True)

In [6]:
pagamentos_pedido_preprocessados.show(n=5, truncate=False)
produtos_preprocessados.show(n=5, truncate=False)
joins_itens_pedido.show(n=5, truncate=False)
avaliacoes_pedido.show(n=5, truncate=False)

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamentos_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333             |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664           |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664           |
|ba7899792

In [7]:
itens_pedido_df = joins_itens_pedido.na.drop()

montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas'
)

itens_pedido_df = montar_vetor.transform(itens_pedido_df)
itens_pedido_df.show(5)

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------+--------------------+---------------

In [8]:
# Definir e treinar o modelo K-Means
kmeans_model = KMeans(featuresCol='caracteristicas', k=3)
modelo_kmeans = kmeans_model.fit(itens_pedido_df)

In [9]:
predicao = modelo_kmeans.transform(itens_pedido_df)
predicao.select('id_produto', 'prediction').show()

+--------------------+----------+
|          id_produto|prediction|
+--------------------+----------+
|4244733e06e7ecb49...|         0|
|ef92defde845ab845...|         0|
|557d850972a7d6f79...|         1|
|d63c1011f49d98b97...|         0|
|3f27ac8e699df3d30...|         0|
|5ed9eaf534f6936b5...|         0|
|553e0e7590d3116a0...|         0|
|5d7c23067ed3fc8c6...|         0|
|5a419dbf24a8c9718...|         0|
|21b1c2f67a9aafb5a...|         2|
|1c0c0093a48f13ba7...|         0|
|89321f94e35fc6d79...|         0|
|38afdf723b95d455b...|         0|
|672e757f331900b9d...|         0|
|28b4eced95a52d9c4...|         0|
|b10eba910a974df70...|         0|
|4089861a1bd4685da...|         0|
|fe59a1e006df3ac42...|         0|
|0b0172eb0fd18479d...|         0|
|0b0172eb0fd18479d...|         0|
+--------------------+----------+
only showing top 20 rows



In [10]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas', predictionCol='prediction', metricName='silhouette')
silhouette = evaluator.evaluate(predicao)
print(f'silhouette = score {silhouette}')

silhouette = score 0.91844438474722


In [12]:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(itens_pedido_df)
itens_pedidos_normalizado_df = modelo_scaler.transform(itens_pedido_df)

kmeans_normalizado_model = KMeans(featuresCol='caracteristicas_normalizado', k=3)
modelo_kmeans_normalizado = kmeans_normalizado_model.fit(itens_pedidos_normalizado_df)

predicao_normalizada = modelo_kmeans_normalizado.transform(itens_pedidos_normalizado_df)
predicao_normalizada.select('id_produto', 'prediction').show()

+--------------------+----------+
|          id_produto|prediction|
+--------------------+----------+
|4244733e06e7ecb49...|         1|
|ef92defde845ab845...|         1|
|557d850972a7d6f79...|         2|
|d63c1011f49d98b97...|         1|
|3f27ac8e699df3d30...|         1|
|5ed9eaf534f6936b5...|         1|
|553e0e7590d3116a0...|         1|
|5d7c23067ed3fc8c6...|         1|
|5a419dbf24a8c9718...|         0|
|21b1c2f67a9aafb5a...|         0|
|1c0c0093a48f13ba7...|         0|
|89321f94e35fc6d79...|         1|
|38afdf723b95d455b...|         1|
|672e757f331900b9d...|         1|
|28b4eced95a52d9c4...|         0|
|b10eba910a974df70...|         0|
|4089861a1bd4685da...|         1|
|fe59a1e006df3ac42...|         1|
|0b0172eb0fd18479d...|         1|
|0b0172eb0fd18479d...|         1|
+--------------------+----------+
only showing top 20 rows



In [13]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas', predictionCol='prediction', metricName='silhouette')
silhouette = evaluator.evaluate(predicao)
print(f'Silhouette Score: {silhouette}')

Silhouette Score: 0.91844438474722


In [14]:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(itens_pedido_df)
itens_pedidos_normalizado_df = modelo_scaler.transform(itens_pedido_df)

kmeans = KMeans(featuresCol='caracteristicas_normalizado', k=3)
modelo_kmeans = kmeans.fit(itens_pedidos_normalizado_df)

predicao_normalizada = modelo_kmeans.transform(itens_pedidos_normalizado_df)
predicao_normalizada.select('id_produto', 'prediction').show()

+--------------------+----------+
|          id_produto|prediction|
+--------------------+----------+
|4244733e06e7ecb49...|         1|
|ef92defde845ab845...|         1|
|557d850972a7d6f79...|         2|
|d63c1011f49d98b97...|         1|
|3f27ac8e699df3d30...|         1|
|5ed9eaf534f6936b5...|         1|
|553e0e7590d3116a0...|         1|
|5d7c23067ed3fc8c6...|         1|
|5a419dbf24a8c9718...|         0|
|21b1c2f67a9aafb5a...|         0|
|1c0c0093a48f13ba7...|         0|
|89321f94e35fc6d79...|         1|
|38afdf723b95d455b...|         1|
|672e757f331900b9d...|         1|
|28b4eced95a52d9c4...|         0|
|b10eba910a974df70...|         0|
|4089861a1bd4685da...|         1|
|fe59a1e006df3ac42...|         1|
|0b0172eb0fd18479d...|         1|
|0b0172eb0fd18479d...|         1|
+--------------------+----------+
only showing top 20 rows



In [15]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas_normalizado', predictionCol='prediction', metricName='silhouette')
silhouette = evaluator.evaluate(predicao_normalizada)
print(f'Silhouette Score: {silhouette}')

Silhouette Score: 0.6346562662751096
